In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch import nn
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")            #comment this line to run with GPU
tpr = torch.hub.load('yangsenius/TransPose:main', 'tpr_a4_256x192', pretrained=True)
#tph = torch.hub.load('yangsenius/TransPose:main', 'tph_a4_256x192', pretrained=True, device=device)


#print(tph)
DATASET_PATH = './dataset/'
positions = os.listdir(DATASET_PATH)
#print(os.listdir(DATASET_PATH+entries[0]))
images = list()

def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

class YogaPoseDataset(Dataset):

    def __init__(self, dataset_path):
        self.data_path = dataset_path
        # call to init the data
        self.size = 256,192
        self._init_data()

    def _init_data(self):
        positions = os.listdir(self.data_path)
        images = list()
        labels = list()
        for idx, position in enumerate(positions):
            for file in os.listdir(DATASET_PATH + position):
                if not file.endswith('.gif'):
                    f = cv2.imread(DATASET_PATH + position + '/' + file,cv2.IMREAD_COLOR)

                    #print(position + '/' + file)
                    #print(f.shape)
                    height,width, channels = f.shape
                    if height > width:
                        scale_percent = 256/height
                        f = cv2.resize(f,(int(width*scale_percent),256))
                        if f.shape[1] > 192:
                            scale_percent = 192/width
                            f = cv2.resize(f,(192,int(height*scale_percent)))
                    else:
                        scale_percent = 192/width
                        f = cv2.resize(f,(192,int(height*scale_percent)))
                        if f.shape[0] > 256:
                            scale_percent = 256/height
                            f = cv2.resize(f,(int(width*scale_percent),256))

                    height,width, channels = f.shape
                    #print(f.shape,"after resize")
                    f = cv2.copyMakeBorder(f, 256-height, 0, 192-width, 0, cv2.BORDER_CONSTANT,value=0)
                    #print(f.shape,"after padding")
                    #if torch.FloatTensor(f.getdata()).size()[1] != 3: f.show()
                    data = torch.reshape(torch.FloatTensor(f).to(device),(3,256,192))
                    #print(data.shape)
                    #print(data.size())
                    ##images.append(data)
                    ##labels.append(torch.tensor(idx))
                    images.append((idx,data))


        #print(images)
        ##images = torch.stack(images)
        ##labels = torch.stack(labels)
        #print(labels)
        #print(images.size())
        ##mask = np.arange(labels.size()[0])
        np.random.shuffle(images)
        self.images = images
        #print(type(images),type(images[0]))
        ##self.images = images[mask]
        ##self.labels = labels[mask]
        #print(self.labels.size())
        #print(self.labels)

    def __len__(self):
        # returns the number of samples in our dataset
        return len(self.images)




    def getData(self):
        return self.images

    #def getY(self):
    #    return self.labels

    def __getitem__(self, idx):
        # returns the idx-th sample
        return self.images[idx]

    def collate_fn(self,data):
        #print(len(data),data)
        #print(data[0].size())
        #print(data[1].size())
        #print('lwwwwwwwwwwwwwww',[x[0].size() for x in data])
        #print(torch.cat([x[0] for x in data]).size(),data[0][0].size())
        #print(torch.stack([x[1] for x in data]).size(),data[0][1],[x[1] for x in data])
        Xs = torch.stack([x[1] for x in data])
        print(Xs.size())
        y = torch.stack([torch.tensor(x[0]) for x in data])
        print(y.size())
        return Xs,y


>>Load pretrained weights from url: https://github.com/yangsenius/TransPose/releases/download/Hub/tp_r_256x192_enc4_d256_h1024_mh8.pth
Successfully loaded model  (on cpu) with pretrained weights!


Using cache found in /home/michele/.cache/torch/hub/yangsenius_TransPose_main
/home/michele/.cache/torch/hub/yangsenius_TransPose_main/lib/models/transpose_r.py:333: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = temperature ** (2 * (dim_t // 2) / one_direction_feats)


In [2]:
dataset = YogaPoseDataset(DATASET_PATH)
split_position = int((len(dataset)//10)*7)
trainset = dataset[:split_position]
testset = dataset[split_position:]
print(trainset)

libpng warning: Ignoring incorrect cHRM white(.34575,.35855) r(.6485,.33088)g(.32121,.59787)b(.15589,.06604) when sRGB is also present
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
dataset[0]

(84,
 tensor([[[  0.,   0.,   0.,  ..., 194., 194., 194.],
          [200., 200., 200.,  ...,  63.,  63.,  63.],
          [ 50.,  50.,  50.,  ..., 180., 180., 180.],
          ...,
          [107., 107., 107.,  ..., 202., 202., 202.],
          [207., 207., 207.,  ..., 180., 180., 180.],
          [  0.,   0.,   0.,  ...,  93.,  93.,  93.]],
 
         [[170., 170., 170.,  ..., 189., 189., 189.],
          [193., 193., 193.,  ..., 198., 198., 198.],
          [  0.,   0.,   0.,  ..., 157., 157., 157.],
          ...,
          [ 84.,  84.,  84.,  ..., 187., 187., 187.],
          [  0.,   0.,   0.,  ..., 160., 160., 160.],
          [154., 154., 154.,  ..., 108., 108., 108.]],
 
         [[ 55.,  55.,  55.,  ..., 188., 188., 188.],
          [  0.,   0.,   0.,  ..., 152., 152., 152.],
          [137., 137., 137.,  ...,  79.,  79.,  79.],
          ...,
          [  0.,   0.,   0.,  ...,  99.,  99.,  99.],
          [ 81.,  81.,  81.,  ..., 148., 148., 148.],
          [120., 120., 120

In [4]:
class PoseClassifier(nn.Module):
    def __init__(self, n_class, fine_tune=False, pretrained=True):
        super(PoseClassifier, self).__init__()

        self.tph = torch.hub.load('yangsenius/TransPose:main', 'tph_a4_256x192', pretrained=True, device=device)
        self.fc1 = nn.Linear(52224,1000).to(device)
        self.fc2 = nn.Linear(1000,n_class).to(device)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = x.to(device)
        out = self.tph(x)
        out = torch.reshape(out,(out.size(0),-1))
        print(out.size())
        out = self.fc1(out)
        #m = nn.BatchNorm1d(1000,device=device)
        #out = m(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [5]:
num_classes = 107
model = PoseClassifier(n_class=num_classes)
num_epochs = 30
batch_size = 32
learning_rate = 1e-3
learning_rate_decay = 0.99
params_to_update = model.parameters()
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

>>Load pretrained weights from url: https://github.com/yangsenius/TransPose/releases/download/Hub/tp_h_48_256x192_enc4_d96_h192_mh1.pth


Using cache found in /home/michele/.cache/torch/hub/yangsenius_TransPose_main
/home/michele/.cache/torch/hub/yangsenius_TransPose_main/lib/models/transpose_h.py:514: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = temperature ** (2 * (dim_t // 2) / one_direction_feats)


Successfully loaded model  (on cpu) with pretrained weights!


In [6]:
model.to(device)
params_to_update = model.parameters()
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params_to_update, lr=learning_rate)
train_loader = torch.utils.data.DataLoader(dataset=trainset,batch_size=batch_size,shuffle=False,collate_fn=dataset.collate_fn)
val_loader = torch.utils.data.DataLoader(dataset=testset,batch_size=batch_size,shuffle=False,collate_fn=dataset.collate_fn)
# Train the model
lr = learning_rate
total_step = len(train_loader)
loss_train = []
loss_val = []
best_accuracy = None
accuracy_val = []

In [7]:
for image,label in train_loader:
    print(image,label)
    break

torch.Size([5, 3, 256, 192])
torch.Size([5])
tensor([[[[  0.,   0.,   0.,  ..., 194., 194., 194.],
          [200., 200., 200.,  ...,  63.,  63.,  63.],
          [ 50.,  50.,  50.,  ..., 180., 180., 180.],
          ...,
          [107., 107., 107.,  ..., 202., 202., 202.],
          [207., 207., 207.,  ..., 180., 180., 180.],
          [  0.,   0.,   0.,  ...,  93.,  93.,  93.]],

         [[170., 170., 170.,  ..., 189., 189., 189.],
          [193., 193., 193.,  ..., 198., 198., 198.],
          [  0.,   0.,   0.,  ..., 157., 157., 157.],
          ...,
          [ 84.,  84.,  84.,  ..., 187., 187., 187.],
          [  0.,   0.,   0.,  ..., 160., 160., 160.],
          [154., 154., 154.,  ..., 108., 108., 108.]],

         [[ 55.,  55.,  55.,  ..., 188., 188., 188.],
          [  0.,   0.,   0.,  ..., 152., 152., 152.],
          [137., 137., 137.,  ...,  79.,  79.,  79.],
          ...,
          [  0.,   0.,   0.,  ...,  99.,  99.,  99.],
          [ 81.,  81.,  81.,  ..., 148., 1

In [8]:
model.to(device)
params_to_update = model.parameters()
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params_to_update, lr=learning_rate)
train_loader = torch.utils.data.DataLoader(dataset=trainset,batch_size=batch_size,shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset=testset,batch_size=batch_size,shuffle=False)
# Train the model
lr = learning_rate
total_step = len(train_loader)
loss_train = []
loss_val = []
best_accuracy = None
accuracy_val = []
#best_model = type(model)(num_classes, fine_tune, pretrained) # get a new instance
for epoch in range(num_epochs):

    model.train()

    loss_iter = 0
    for i, (labels, images) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_iter += loss.item()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    loss_train.append(loss_iter/(len(train_loader)*batch_size))


    # Code to update the lr
    lr *= learning_rate_decay
    update_lr(optimizer, lr)


    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        loss_iter = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            loss_iter += loss.item()

        loss_val.append(loss_iter/(len(val_loader)*batch_size))

        accuracy = 100 * correct / total
        accuracy_val.append(accuracy)

        print('Validataion accuracy is: {} %'.format(accuracy))
        #################################################################################
        # TODO: Q2.b Use the early stopping mechanism from previous questions to save   #
        # the model with the best validation accuracy so-far (use best_model).          #
        #################################################################################

        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        early_stop = False
        patience = 3
        if epoch > patience - 1:
            for j in range(patience - 1):
                if max(accuracy_val) > list(reversed(accuracy_val))[j]:
                    if "not_improving_epochs" in locals(): not_improving_epochs += 1
                    else: not_improving_epochs = 1
                    print('Not saving the model')
                else:
                    not_improving_epochs = 0
                    best_model = model
                    print("Saving the model")
                    break
                if not_improving_epochs >= patience:
                    early_stop = True
                    print('Early stopping')
                    break
                break

RuntimeError: CUDA out of memory. Tried to allocate 180.00 MiB (GPU 0; 7.79 GiB total capacity; 4.63 GiB already allocated; 110.25 MiB free; 5.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF